In [1]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [2]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator as Generator 
import model_definitions

### Open input file

In [3]:
SEQUENCE_LENGTH=20
ONE_CAR=True
PATH=r"D:\Users\Students\Daniele\records\OneCar_db.txt"
#PATH=r"C:\Users\dansp\OneDrive\Desktop\Tesi\SCRITTURA\OneCar_tack1_db.txt"
BATCH_SIZE=256*2
FEATURES=7 
FEATURES_PREDICTED=6
DISCARD=4

In [4]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [5]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
Generator.DISCARD=DISCARD
train,val,test=Generator.single_care_dataframe(PATH)

In [6]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

def SubtractAndBuildDF(df):
    dfs=[]
    for i in range(len(df)):
        dfs.append(SubtractDF(df[i]))
        #dfs.append(df[i])
    united=dfs[0]
    for i in range(1,len(df)):
        united=pd.concat([united,dfs[i] ],ignore_index=True)
    return united

df_train= SubtractAndBuildDF(train)
df_val= SubtractAndBuildDF(val)
df_test= SubtractAndBuildDF(test)

In [7]:
df_train.head()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
0,0.00000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,4,32,0.000000,0.000000,0.1,0,0
1,-0.00005,0.09370,0.000000,1.831339,0.0000,0.000000,0.000000,21.263850,4,32,-0.124001,0.390583,0.1,0,0
2,-0.00061,0.29761,-0.030470,1.845064,-0.0278,-0.024275,-1.523513,-5.043560,4,32,-0.124009,0.394551,0.1,0,0
3,-0.11072,0.43237,-0.966261,1.363851,-3.8248,-0.440160,22.816403,4.689970,4,32,-0.125485,0.400316,0.1,0,0
4,-0.09253,0.55347,0.593763,0.755228,-1.9178,0.321507,12.455210,-18.515107,4,32,-0.126719,0.407695,0.1,0,0


In [8]:
#df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [9]:
#df_train.head()

In [14]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,10000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,2600)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,2600)


Length: 2720316 races: 84 n batches: 2711916 / 512
Length: 1286892 races: 42 n batches: 1282692 / 512
Length: 1303610 races: 42 n batches: 1299410 / 512


In [10]:
#x_y=gen_test[0]
#print(x_y[0][0])
#x_y[1][0]


In [11]:
#x_y[0][0]

### Model definition

#### Train and print

In [12]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [14]:
NAME=f"MLP_DISCARD_{DISCARD}_DELTA_LONG_TRAIN"
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=1000,verbose=1,name=NAME,path="logs")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/{NAME}')

Epoch 1/10
5158/5158 [==============================] - 1237s 240ms/step - loss: 12.4075 - mean_absolute_error: 1.3992 - accuracy: 0.0000e+00 - val_loss: 1.7049 - val_mean_absolute_error: 0.8486 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/10
5158/5158 [==============================] - 1758s 341ms/step - loss: 1.3769 - mean_absolute_error: 0.7643 - accuracy: 0.0000e+00 - val_loss: 1.2612 - val_mean_absolute_error: 0.7272 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5158/5158 [==============================] - 1663s 322ms/step - loss: 1.2098 - mean_absolute_error: 0.7139 - accuracy: 0.0000e+00 - val_loss: 1.2066 - val_mean_absolute_error: 0.7096 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5158/5158 [==============================] - 1689s 327ms/step - loss: 1.1864 - mean_absolute_error: 0.7058 - accuracy: 0.0000e+00 - val_loss: 1.1945 - val_mean_absolute_error: 0.7048 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
5158/5158 [=========================

In [13]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

Epoch 1/10
5655/5655 [==============================] - 2340s 413ms/step - loss: 0.9373 - mean_absolute_error: 0.6307 - accuracy: 4.6051e-08 - val_loss: 0.7536 - val_mean_absolute_error: 0.5764 - val_accuracy: 9.2807e-08 - lr: 1.0000e-04
Epoch 2/10
5655/5655 [==============================] - 2301s 407ms/step - loss: 0.5683 - mean_absolute_error: 0.5047 - accuracy: 0.0000e+00 - val_loss: 0.4222 - val_mean_absolute_error: 0.4416 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5655/5655 [==============================] - 2333s 412ms/step - loss: 0.3508 - mean_absolute_error: 0.4066 - accuracy: 0.0000e+00 - val_loss: 0.2898 - val_mean_absolute_error: 0.3652 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5655/5655 [==============================] - 2304s 407ms/step - loss: 0.2785 - mean_absolute_error: 0.3564 - accuracy: 0.0000e+00 - val_loss: 0.2501 - val_mean_absolute_error: 0.3341 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
5655/5655 [==========================

INFO:tensorflow:Assets written to: saved_model/CNN_LSTM32_1_32_PosAssoluta\assets


INFO:tensorflow:Assets written to: saved_model/CNN_LSTM32_1_32_PosAssoluta\assets


In [15]:
lr =1e-4
units=64
cells=3
normalization=False
mlp_units=64
mlp_cells=12
name=f"LSTM{cells}_{units}_{lr}_{DISCARD}_MLP_{mlp_cells}_{mlp_units}"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(
    gen_train,
    gen_val,
    learning_rate=lr,
    patience=6,
    name=name,
    path=f"new_split_Tile_info_DISCARD{DISCARD}",
    epoch=100, 
    dropout=0.2, 
    cells=cells,
    units=units,
    mlp_units=mlp_units,
    mlp_cells=mlp_cells,
    normalization=normalization,
    verbose=1
)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
#tf.saved_model.save(lstm_ph,f'saved_model/{name}')

Epoch 1/100
5293/5294 [============================>.] - ETA: 0s - loss: 1.5367 - mean_absolute_error: 0.6437 - accuracy: 1.2300e-07

INFO:tensorflow:Assets written to: ./new_split_Tile_info_DISCARD4/LSTM3_64_0.0001_4_MLP_12_64/tmp\checkpoint\assets


INFO:tensorflow:Assets written to: ./new_split_Tile_info_DISCARD4/LSTM3_64_0.0001_4_MLP_12_64/tmp\checkpoint\assets


5294/5294 [==============================] - 606s 114ms/step - loss: 1.5366 - mean_absolute_error: 0.6436 - accuracy: 1.2298e-07 - val_loss: 2.6063 - val_mean_absolute_error: 0.8054 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/100
5293/5294 [============================>.] - ETA: 0s - loss: 0.7148 - mean_absolute_error: 0.4549 - accuracy: 1.8450e-07

INFO:tensorflow:Assets written to: ./new_split_Tile_info_DISCARD4/LSTM3_64_0.0001_4_MLP_12_64/tmp\checkpoint\assets


...

INFO:tensorflow:Assets written to: ./new_split_Tile_info_DISCARD4/LSTM3_64_0.0001_4_MLP_12_64/tmp\checkpoint\assets


5294/5294 [==============================] - 590s 111ms/step - loss: 0.3244 - mean_absolute_error: 0.2742 - accuracy: 9.2233e-07 - val_loss: 0.7694 - val_mean_absolute_error: 0.3971 - val_accuracy: 2.6010e-07 - lr: 1.0000e-04
Epoch 32/100
5294/5294 [==============================] - 583s 110ms/step - loss: 0.3247 - mean_absolute_error: 0.2737 - accuracy: 6.1489e-07 - val_loss: 0.7941 - val_mean_absolute_error: 0.3974 - val_accuracy: 5.2021e-07 - lr: 1.0000e-04
Epoch 33/100
5294/5294 [==============================] - 586s 111ms/step - loss: 0.3220 - mean_absolute_error: 0.2725 - accuracy: 5.5340e-07 - val_loss: 0.7856 - val_mean_absolute_error: 0.4029 - val_accuracy: 2.6010e-07 - lr: 1.0000e-04
Epoch 34/100
5294/5294 [==============================] - ETA: 0s - loss: 0.3205 - mean_absolute_error: 0.2719 - accuracy: 5.5340e-07

INFO:tensorflow:Assets written to: ./new_split_Tile_info_DISCARD4/LSTM3_64_0.0001_4_MLP_12_64/tmp\checkpoint\assets


INFO:tensorflow:Assets written to: ./new_split_Tile_info_DISCARD4/LSTM3_64_0.0001_4_MLP_12_64/tmp\checkpoint\assets


5294/5294 [==============================] - 594s 112ms/step - loss: 0.3205 - mean_absolute_error: 0.2719 - accuracy: 5.5340e-07 - val_loss: 0.7129 - val_mean_absolute_error: 0.3760 - val_accuracy: 1.3005e-07 - lr: 1.0000e-04
Epoch 35/100
5294/5294 [==============================] - 582s 110ms/step - loss: 0.3193 - mean_absolute_error: 0.2713 - accuracy: 7.9935e-07 - val_loss: 0.7733 - val_mean_absolute_error: 0.3976 - val_accuracy: 7.8031e-07 - lr: 1.0000e-04
Epoch 36/100
5294/5294 [==============================] - 585s 110ms/step - loss: 0.3181 - mean_absolute_error: 0.2704 - accuracy: 7.3786e-07 - val_loss: 0.8394 - val_mean_absolute_error: 0.4092 - val_accuracy: 5.2021e-07 - lr: 1.0000e-04
Epoch 37/100
5294/5294 [==============================] - 586s 111ms/step - loss: 0.3148 - mean_absolute_error: 0.2695 - accuracy: 6.7638e-07 - val_loss: 0.8305 - val_mean_absolute_error: 0.4148 - val_accuracy: 5.2021e-07 - lr: 1.0000e-04
Epoch 38/100
5294/5294 [==============================] -

In [ ]:
test=0
for batch in gen_val:
    print(test)
    test+=1
test

In [56]:
test[1].shape

IndexError: list index out of range

In [93]:
y_pred = lstm_ph.predict(gen_test)


47/47 [==============================] - 7s 151ms/step


In [94]:
y_pred

array([[ 1.4616523 , -0.7965714 , -2.364098  ],
       [ 1.4196657 , -0.72882974, -2.1959066 ],
       [ 1.3491808 , -0.6210692 , -2.2700336 ],
       ...,
       [ 0.10450363,  0.48482472, -0.6983368 ],
       [ 0.10220872,  0.48504275, -0.7115003 ],
       [ 0.09761263,  0.49852407, -0.72805303]], dtype=float32)

In [95]:
ev

[2.1729495525360107, 0.8310683965682983, 0.0]

In [27]:
lr =1e-3
units=512
cells=8
normalization=False
name=f"LSTM{cells}_{units}_{lr}_DISCARD_{DISCARD}_DELTA"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=50,name=name,path="logs",
                                                     epoch=100, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

INFO:tensorflow:Assets written to: saved_model/LSTM8_512_0.001_DISCARD_19_DELTA\assets


INFO:tensorflow:Assets written to: saved_model/LSTM8_512_0.001_DISCARD_19_DELTA\assets


In [29]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

INFO:tensorflow:Assets written to: saved_model/LSTM4_128_0.001_DISCARD_19_DELTA\assets


INFO:tensorflow:Assets written to: saved_model/LSTM4_128_0.001_DISCARD_19_DELTA\assets


In [12]:
model=model_definitions.MLP_Model(32)
model.load_weights("saved_model\MLP0001_PosAssoluta")

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 140)               0         
                                                                 
 dense (Dense)               (None, 32)                4512      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 4,611
Trainable params: 4,611
Non-trainable params: 0
_________________________________________________________________


In [14]:
zero=gen_test[0]
toP=zero[0][0]
pred=zero[1][0]
print(toP)
print(pred)
model.predict(toP)

tf.Tensor(
[[-2.156165e+01 -1.154109e+01  3.978183e-01  1.900000e+01  2.000000e+01
   2.002995e-01  5.461393e-01]
 [-2.191261e+01 -1.069794e+01  9.277464e-01  1.900000e+01  2.000000e+01
   1.828646e-01  5.285156e-01]
 [-2.236745e+01 -9.373469e+00  1.181234e+00  1.900000e+01  2.000000e+01
   1.650993e-01  5.127604e-01]
 [-2.271277e+01 -8.215466e+00  8.408008e-01  1.900000e+01  2.000000e+01
   1.469889e-01  4.992383e-01]
 [-2.317203e+01 -7.699648e+00  2.726782e-01  1.900000e+01  2.000000e+01
   1.285677e-01  4.866276e-01]
 [-2.354665e+01 -7.331343e+00  8.814032e-02  1.900000e+01  2.000000e+01
   1.098258e-01  4.747331e-01]
 [-2.385743e+01 -7.165668e+00  3.089963e-02  1.900000e+01  2.000000e+01
   9.082031e-02  4.631966e-01]
 [-2.413132e+01 -7.114067e+00  1.184550e-02  1.900000e+01  2.000000e+01
   7.158691e-02  4.517969e-01]
 [-2.438515e+01 -7.120355e+00  4.894121e-03  1.900000e+01  2.000000e+01
   5.214518e-02  4.404102e-01]
 [-2.460253e+01 -6.905964e+00  1.876919e-01  1.900000e+01  2.0

ValueError: in user code:

    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequential" is 

In [ ]:
import tf2onnx
import onnx
input_signature=(tf.TensorSpec([None,SEQUENCE_LENGTH,FEATURES],tf.float32,name='x'))
onnx_model,_=tf2onnx.converter.from_keras(model,input_signature,opset=9)

In [49]:
races=united["RACE"].max()
print(races*0.5)
training=united.loc[united['RACE'] <= races*0.5]
training.tail(10)

83.5


,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
5051758,-5.32727,2.571900,4.838010,5.90930,13.9886,0.747973,27.840900,24.523110,4,32,0.456929,0.080146,0.2,83,9
5051759,-4.23840,3.754450,4.522620,4.17718,14.0799,-0.201847,-13.900470,-23.734710,4,32,0.400417,0.130206,0.2,83,9
5051760,-3.61328,4.332640,1.717370,2.05974,4.8036,-1.075305,-18.862057,-9.560299,4,32,0.352240,0.187974,0.2,83,9
5051761,-3.38379,4.573670,1.456980,0.93513,2.5532,0.547570,12.505337,0.734615,4,32,0.307122,0.248957,0.2,83,9
5051762,-3.03601,4.786750,1.265280,0.82554,3.1718,-0.565750,-4.297250,-2.067471,4,32,0.266642,0.312780,0.2,83,9
5051763,-3.02600,4.787650,0.228000,-0.06486,-0.8624,0.594987,2.488040,-1.159763,4,32,0.226296,0.376615,0.2,83,9
5051764,-2.67359,4.945070,2.600840,1.12468,7.7518,0.070049,5.761860,3.362847,4,32,0.190648,0.442550,0.2,83,9
5051765,-1.90893,5.190311,4.206075,1.07436,9.8041,-0.011295,-8.786300,-4.183674,16,0,0.165195,-0.476492,0.2,83,9
5051766,-1.24964,5.344787,2.852189,0.50855,6.1715,0.238537,11.289770,-2.462674,16,0,0.148534,-0.333965,0.2,83,9
5051767,-0.57885,5.431946,3.084546,0.64175,5.5514,-0.873222,-14.921353,2.689744,16,0,0.140815,-0.189113,0.2,83,9


In [25]:
t=df_train["ROT"].idxmax()
df_train.iloc[t-2:t+3]

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
2470682,0.00270,-4.962400,0.123759,-0.813940,-0.0700,0.055793,-1.702185,-0.837422,17,22,-0.459295,-0.110540,0.2,5,9
2470683,0.00250,-5.130860,-0.017524,-0.787650,0.0002,0.000706,0.121586,0.341511,17,22,-0.459261,-0.178952,0.2,5,9
2470684,-343.65538,23.621341,-0.006691,26.007860,179.8327,-0.145945,0.091397,3.962040,16,0,-0.041333,-0.228001,0.2,5,9
2470685,-0.00781,0.513184,-0.092801,4.798755,-0.9364,0.148455,-0.496246,25.744220,16,0,-0.041437,-0.214316,0.2,5,9
2470686,-0.03454,1.323730,-0.101864,3.123408,-0.3048,-0.017739,0.488042,-7.024400,16,0,-0.041898,-0.179017,0.2,5,9


In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf


SEQUENCE_LENGTH=20
ONLY_ONE_CAR=True
CARS=4
FEATURES=5
DISCARD=9
COLUMNS=["Player", "X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC_X","ACC_Z","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME"]
C_NoPLayer=["X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC_X","ACC_Z","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME","RACE","GROUP"]

def normalize_df(df,minimum,maximum):
    normalized=(df-minimum)/(maximum-minimum)
    normalized["RACE"]=df["RACE"]
    return normalized

      
    
def single_care_dataframe(path):
    df_gara=pd.read_csv(path, sep=";", header=None , decimal=',',names=COLUMNS)
    #create new column called race
    df_gara["RACE"]=0
    # fill race column based on the cumulative sum of rows starting with '_'
    #idx_gara stores each row wich starts with '_'
    idx_gara=(df_gara[df_gara["Player"].str.startswith("_")].index)
    df_gara.loc[idx_gara,"Player"]=df_gara.loc[idx_gara,"Player"].str.replace("_","")#replace name without the _

    df_gara.loc[idx_gara,"RACE"]=1
    df_gara.RACE=df_gara.RACE.cumsum()
    #Create new column Length wich specifiens the total length of a race
    df_gara["LENGTH"]=df_gara.groupby("RACE")["Player"].transform("count")
    #if race is lewer then a minimum then it is discarded
    df_races=df_gara.query(f"LENGTH > {SEQUENCE_LENGTH*DISCARD+1}").reset_index(drop=True)
    df_races.drop(["Player","LENGTH"],axis=1,inplace=True)
    print(df_races["RACE"].nunique())
    print(df_races["RACE"].max())

    train,val,test = split_train_validation_test(
        df_races,
        "RACE"
    ) 
    df_train= divide_into_groups(train)
    df_val= divide_into_groups(val)
    df_test= divide_into_groups(test)

    return df_train, df_val,df_test

def divide_into_groups(df_x):
    df=df_x.copy()
    dfs=[]
    for i in range(DISCARD+1):
        df["GROUP"]=i
        temp=df.iloc[i::DISCARD+1]
        temp.reset_index(drop=True,inplace=True)
        dfs.append(temp)
    return dfs

def subtraction_columns(df):
    df_copy=df.shift(1,fill_value=0)
    cols = df.columns.difference(['RACE','GROUP',"TILE","TILE_IND","X_RELATIVE","Z_RELATIVE"])
    df[cols] = df[cols].sub(df_copy[cols])
    df["ROT"]=(df["ROT"]+180)%360-180
    df.iloc[0,:-7]=0
    df.iloc[0,10:12]=0
    df.iloc[0,-3]=df.iloc[1,-3]
    
    return df

def get_split(x,first,second):
    first =int(x.shape[0]*first)
    second = int(x.shape[0]*second) 
    return x[first:second]
    
def split_train_validation_test(df,group_col,train_split=0.5,val_split=0.25,test_split=0.25):
    val_split +=train_split
    
    if val_split >1:
        raise ValueError(
            f"Train + Validation split cannot be higher tan 1 given {val_split}"
        )

    races=df["RACE"].max()+1
    df_train=df.loc[df['RACE'] < races*train_split]
    df_val=df.loc[(df['RACE'] >= races*train_split) & (df['RACE'] < races*val_split)]
    df_test=df.loc[df['RACE'] >= races*val_split]
    
    #group by race (and player name for single car) and create a new array containing foreach race a dataset
    #df_train= df.groupby(group_col,group_keys=False).apply(get_split, first = 0, second= train_split)
    #df_val= df.groupby(group_col,group_keys=False).apply(get_split, first = train_split, second= val_split)
    #df_test= df.groupby(group_col,group_keys=False).apply(get_split, first = val_split, second= 1)
    #
    #
    ##since each race was plittend into train,val and test the result of previous operation is an array containing the data 
    ##foreach race, therefore to have the end dataframe we must concatenate each element
    #df_train=recreate_dataframe(df_train)
    #df_val=recreate_dataframe(df_val)
    #df_test=recreate_dataframe(df_test)
    
    return df_train, df_val,df_test

def recreate_dataframe(series):
    columns=C_NoPLayer 
    series.columns=columns
    df=series.reset_index(drop=True)
    #v#alues=series.values
    #df=pd.DataFrame(values[0],columns=columns)
    #
#
    #for serie in values[1:]:
    #    df= pd.concat([df, pd.DataFrame(serie,columns=columns)],ignore_index=True)
    return df

def batch_generator(df):
    
    #crea un nuovo dataframe con sequence_length elementi per un numero di volte pari al batch
    dropped_df=df.drop(["TIME","RACE","GROUP","X","Z","ACC_X","ACC_Z","ANG_VEL_Y"],axis=1).reset_index(drop=True)
    #dropped_df["ROT"]=dropped_df["ROT"]/360.0;
    target_df=dropped_df.drop(["TILE","TILE_IND","X_RELATIVE","Z_RELATIVE"],axis=1).reset_index(drop=True)
    #dropped_df=dropped_df.drop([ "VEL_X","VEL_Z","ROT"],axis=1).reset_index(drop=True)
    for i in range(len(dropped_df)-SEQUENCE_LENGTH):
        inputs=np.array(dropped_df.loc[i:SEQUENCE_LENGTH-1+i,:].values)
        targets=target_df.iloc[SEQUENCE_LENGTH+i,:].values
        yield inputs,targets  

def Generator(df):
    grouped=df.groupby(["RACE","GROUP"],group_keys=False).apply(batch_generator)
    for group in grouped:
        for single in group:
            yield single
            
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,batch_size,df,max_batch):
        self.batch_size=batch_size
        self.df=df
        length=df.groupby(["RACE","GROUP"]).apply(countSize)
        
        self.df_length=len(df.index)-(df["RACE"].nunique()*SEQUENCE_LENGTH*CARS)
        self.max_batch=max_batch            
        print(f'Length: {len(df.index)} races: {df["RACE"].nunique()} n batches: {self.df_length} / {batch_size}')
        self.on_epoch_end()
        #self.generator=generator_function(sequence_length,path,totFiles)
        
    def __getitem__(self,index):
        X=[]
        Y=[]
        for i in range(self.batch_size):
            #while True:
            #    x,y=next(self.generator)
            #    x_shape=np.shape(x)
            #    if x_shape[0]==x_shape[1]:
            #        break
            x,y=next(self.generator)
            X.append(x)
            Y.append(y)
            
        #print(np.shape(X))    
        tensor_x=tf.constant(X)
        tensor_y=tf.constant(Y)
        return tensor_x,tensor_y
    
    def __len__(self):
        value=int(self.df_length/self.batch_size-1)
        if value>self.max_batch:
            value=self.max_batch
        return value
    
    def on_epoch_end(self):
        self.generator=Generator(self.df)

In [59]:
#(len(df_train.index) - df_train["RACE"].nunique()*SEQUENCE_LENGTH)/BATCH_SIZE
def countSize(df):
    return len(df.reset_index(drop=True))-SEQUENCE_LENGTH

somma=df_train.groupby(["RACE","GROUP"]).apply(countSize)
somma.sum()

2703516

In [29]:
df_train.tail()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
2720311,-3.02600,4.787650,0.228000,-0.06486,-0.8624,0.594987,2.488040,-1.159763,4,32,0.226296,0.376615,0.2,83,9
2720312,-2.67359,4.945070,2.600840,1.12468,7.7518,0.070049,5.761860,3.362847,4,32,0.190648,0.442550,0.2,83,9
2720313,-1.90893,5.190311,4.206075,1.07436,9.8041,-0.011295,-8.786300,-4.183674,16,0,0.165195,-0.476492,0.2,83,9
2720314,-1.24964,5.344787,2.852189,0.50855,6.1715,0.238537,11.289770,-2.462674,16,0,0.148534,-0.333965,0.2,83,9
2720315,-0.57885,5.431946,3.084546,0.64175,5.5514,-0.873222,-14.921353,2.689744,16,0,0.140815,-0.189113,0.2,83,9
